In [28]:
import pandas as pd

pop_data = pd.read_csv("../3_데이터_자료/대한민국 전체 인구.csv")
pop_region = pd.read_csv("../4_데이터_전처리_자료/시,군,구별 전체 인구 합계 최종.csv")
pop_region_65_over = pd.read_csv("../4_데이터_전처리_자료/시,군,구별 65세 이상 고령자 및 독거 노인 최종.csv")
kiosk = pd.read_csv("../4_데이터_전처리_자료/무인민원발급기 설치 정보 최종.csv")

In [29]:
pop_data.head()

,행정구역별(읍면동),연령별,2024
0,행정구역별(읍면동),연령별,총인구(명)
1,전국,합계,51805547


In [30]:
pop_region.head()

,인구 수,행정구역
0,144486,서울특별시 종로구
1,126899,서울특별시 중구
2,203401,서울특별시 용산구
3,274688,서울특별시 성동구
4,345130,서울특별시 광진구


In [31]:
pop_region_65_over.head()

,행정구역,고령자 수,독거 노인 수
0,서울특별시 종로구,27382,6118
1,서울특별시 중구,24360,6187
2,서울특별시 용산구,36473,7441
3,서울특별시 성동구,48758,9647
4,서울특별시 광진구,56766,11148


In [32]:
kiosk.head()

,행정구역,설치대수
0,서울특별시 종로구,28
1,서울특별시 중구,31
2,서울특별시 용산구,24
3,서울특별시 성동구,30
4,서울특별시 광진구,30


In [33]:
df_r  = pop_region.copy()
df_r6  = pop_region_65_over.copy()
df_k = kiosk.copy()

# 행정구역 이름 통일(세종특별자치시 세종시 -> 세종특별자치시)
df_r["행정구역"]  = df_r["행정구역"].replace({"세종특별자치시 세종시": "세종특별자치시"})
df_r6["행정구역"] = df_r6["행정구역"].replace({"세종특별자치시 세종시": "세종특별자치시"})

# df_r 기준으로 행정구역 키로 df_r6, df_k 병합
df = df_r.merge(df_r6, on="행정구역", how="left").merge(df_k, on="행정구역", how="left")

df["설치대수"] = df["설치대수"].astype(int)

df.head()

,인구 수,행정구역,고령자 수,독거 노인 수,설치대수
0,144486,서울특별시 종로구,27382,6118,28
1,126899,서울특별시 중구,24360,6187,31
2,203401,서울특별시 용산구,36473,7441,24
3,274688,서울특별시 성동구,48758,9647,30
4,345130,서울특별시 광진구,56766,11148,30


In [34]:
# 디지털 과잉 지수 산출을 위한 비율 계산
df["고령인구구성비율"] = df["고령자 수"] / df["인구 수"]
df["독거노인구성비율"] = df["독거 노인 수"] / df["인구 수"]
df["무인행정공급비율"] = df["설치대수"] / df["인구 수"]

df_ratio_1 = df[["행정구역", "고령인구구성비율", "독거노인구성비율", "무인행정공급비율"]].copy()
df_ratio_1.head(229)

# 26.32 21.09

,행정구역,고령인구구성비율,독거노인구성비율,무인행정공급비율
0,서울특별시 종로구,0.189513,0.042343,0.000194
1,서울특별시 중구,0.191964,0.048755,0.000244
2,서울특별시 용산구,0.179316,0.036583,0.000118
3,서울특별시 성동구,0.177503,0.035120,0.000109
4,서울특별시 광진구,0.164477,0.032301,0.000087
...,...,...,...,...
224,경상남도 함양군,0.374676,0.120463,0.000200
225,경상남도 거창군,0.305978,0.091826,0.000291
226,경상남도 합천군,0.427189,0.147599,0.000539
227,제주특별자치도 제주시,0.162283,0.034211,0.000105


In [35]:
# pop는 대한민국 전체 인구 수
pop = 51805547

# 무인 행정 수급 불균형 지수 산출을 위한 비율 계산
df["행정수요비율"] = df["인구 수"] / pop
df["무인행정공급비율"] = df["설치대수"] / df["인구 수"]

df_ratio_2 = df[["행정구역", "행정수요비율", "무인행정공급비율"]].copy()
df_ratio_2.head(229)

,행정구역,행정수요비율,무인행정공급비율
0,서울특별시 종로구,0.002789,0.000194
1,서울특별시 중구,0.002450,0.000244
2,서울특별시 용산구,0.003926,0.000118
3,서울특별시 성동구,0.005302,0.000109
4,서울특별시 광진구,0.006662,0.000087
...,...,...,...
224,경상남도 함양군,0.000677,0.000200
225,경상남도 거창군,0.001127,0.000291
226,경상남도 합천군,0.000752,0.000539
227,제주특별자치도 제주시,0.009534,0.000105


In [36]:
cols = ["고령인구구성비율", "독거노인구성비율", "무인행정공급비율"]

# 디지털 과잉 지수 산출을 위한 비율 정규화 계산
df_norm_1 = df_ratio_1.copy()
df_norm_1[cols] = (df_norm_1[cols] - df_norm_1[cols].min()) / (df_norm_1[cols].max() - df_norm_1[cols].min())
df_norm_1 = df_norm_1.rename(columns = {"고령인구구성비율":"고령인구구성비율(정규화)", 
                                        "독거노인구성비율":"독거노인구성비율(정규화)", 
                                        "무인행정공급비율":"무인행정공급비율(정규화)"})
df_norm_1.head(229)

,행정구역,고령인구구성비율(정규화),독거노인구성비율(정규화),무인행정공급비율(정규화)
0,서울특별시 종로구,0.260953,0.179584,0.221028
1,서울특별시 중구,0.267902,0.229563,0.292546
2,서울특별시 용산구,0.232034,0.134686,0.113682
3,서울특별시 성동구,0.226894,0.123282,0.101249
4,서울특별시 광진구,0.189954,0.101309,0.069680
...,...,...,...,...
224,경상남도 함양군,0.786048,0.788486,0.229233
225,경상남도 거창군,0.591232,0.565276,0.358885
226,경상남도 합천군,0.934967,1.000000,0.710143
227,제주특별자치도 제주시,0.183732,0.116196,0.095680


In [37]:
cols = ["행정수요비율", "무인행정공급비율"]

# 무인 행정 수급 불균형 지수 산출을 위한 비율 정규화 계산
df_norm_2 = df_ratio_2.copy()
df_norm_2[cols] = (df_norm_2[cols] - df_norm_2[cols].min()) / (df_norm_2[cols].max() - df_norm_2[cols].min())
df_norm_2 = df_norm_2.rename(columns = {"행정수요비율":"행정수요비율(정규화)", 
                                        "무인행정공급비율":"무인행정공급비율(정규화)"})
df_norm_2.head(229)

,행정구역,행정수요비율(정규화),무인행정공급비율(정규화)
0,서울특별시 종로구,0.111776,0.221028
1,서울특별시 중구,0.097318,0.292546
2,서울특별시 용산구,0.160207,0.113682
3,서울특별시 성동구,0.218809,0.101249
4,서울특별시 광진구,0.276716,0.069680
...,...,...,...
224,경상남도 함양군,0.021832,0.229233
225,경상남도 거창군,0.041002,0.358885
226,경상남도 합천군,0.025019,0.710143
227,제주특별자치도 제주시,0.399020,0.095680


In [38]:
df_final_1 = df_norm_1.copy()

# 디지털 과잉 지수 산출
df_final_1["디지털과잉지수"] = df_norm_1["고령인구구성비율(정규화)"] * df_norm_1["독거노인구성비율(정규화)"] * df_norm_1["무인행정공급비율(정규화)"]
df_final_1 = df_final_1[["행정구역", "디지털과잉지수"]].copy()
df_final_1.head(229)

,행정구역,디지털과잉지수
0,서울특별시 종로구,0.010358
1,서울특별시 중구,0.017992
2,서울특별시 용산구,0.003553
3,서울특별시 성동구,0.002832
4,서울특별시 광진구,0.001341
...,...,...
224,경상남도 함양군,0.142076
225,경상남도 거창군,0.119943
226,경상남도 합천군,0.663960
227,제주특별자치도 제주시,0.002043


In [39]:
df_final_1["디지털과잉지수"].describe()

count    229.000000
mean       0.084464
std        0.132964
min        0.000000
25%        0.002165
50%        0.011121
75%        0.122253
max        0.663960
Name: 디지털과잉지수, dtype: float64

In [40]:
df_final_2 = df_norm_2.copy()

# 무인 행정 수급 불균형 지수 산출
df_final_2["무인행정수급불균형지수"] = df_norm_2["행정수요비율(정규화)"] - df_norm_2["무인행정공급비율(정규화)"]
df_final_2 = df_final_2[["행정구역", "무인행정수급불균형지수"]].copy()
df_final_2.head(229)

,행정구역,무인행정수급불균형지수
0,서울특별시 종로구,-0.109253
1,서울특별시 중구,-0.195228
2,서울특별시 용산구,0.046525
3,서울특별시 성동구,0.117560
4,서울특별시 광진구,0.207036
...,...,...
224,경상남도 함양군,-0.207401
225,경상남도 거창군,-0.317883
226,경상남도 합천군,-0.685124
227,제주특별자치도 제주시,0.303339


In [41]:
df_final_2["무인행정수급불균형지수"].describe()

count    229.000000
mean      -0.051333
std        0.347891
min       -0.988212
25%       -0.332192
50%       -0.041237
75%        0.207036
max        0.957466
Name: 무인행정수급불균형지수, dtype: float64

In [42]:
thr = df_final_1["디지털과잉지수"].quantile(0.95)
top = df_final_1[df_final_1["디지털과잉지수"] >= thr].sort_values("디지털과잉지수", ascending=False)
top

,행정구역,디지털과잉지수
226,경상남도 합천군,0.663960
197,경상북도 의성군,0.598625
200,경상북도 영덕군,0.565261
217,경상남도 의령군,0.551139
158,전북특별자치도 진안군,0.549492
174,전라남도 보성군,0.465155
173,전라남도 고흥군,0.459488
199,경상북도 영양군,0.455765
222,경상남도 하동군,0.412041
128,충청북도 보은군,0.411476


In [43]:
thr = df_final_1["디지털과잉지수"].quantile(0.05)
bottom = df_final_1[df_final_1["디지털과잉지수"] <= thr].sort_values("디지털과잉지수", ascending=False)
bottom

,행정구역,디지털과잉지수
94,경기도 용인시,0.000150
23,서울특별시 송파구,0.000144
73,울산광역시 북구,0.000119
89,경기도 오산시,0.000111
56,인천광역시 서구,0.000104
98,경기도 김포시,0.000071
68,대전광역시 유성구,0.000069
75,세종특별자치시,0.000059
90,경기도 시흥시,0.000048
52,인천광역시 연수구,0.000018


In [44]:
thr = df_final_2["무인행정수급불균형지수"].quantile(0.95)
top = df_final_2[df_final_2["무인행정수급불균형지수"] >= thr].sort_values("무인행정수급불균형지수", ascending=False)
top

,행정구역,무인행정수급불균형지수
76,경기도 수원시,0.957466
94,경기도 용인시,0.857253
85,경기도 고양시,0.832260
99,경기도 화성시,0.749121
77,경기도 성남시,0.687946
216,경상남도 창원시,0.675515
80,경기도 부천시,0.613597
127,충청북도 청주시,0.603845
84,경기도 안산시,0.520184
88,경기도 남양주시,0.518692


In [45]:
thr = df_final_2["무인행정수급불균형지수"].quantile(0.05)
bottom = df_final_2[df_final_2["무인행정수급불균형지수"] <= thr].sort_values("무인행정수급불균형지수", ascending=False)
bottom

,행정구역,무인행정수급불균형지수
197,경상북도 의성군,-0.606530
208,경상북도 울릉군,-0.611865
222,경상남도 하동군,-0.616996
171,전라남도 곡성군,-0.640601
116,강원특별자치도 영월군,-0.643559
200,경상북도 영덕군,-0.664015
217,경상남도 의령군,-0.681489
226,경상남도 합천군,-0.685124
128,충청북도 보은군,-0.687948
147,충청남도 청양군,-0.688999


In [46]:
df_final_1.to_csv("../5_데이터_처리_자료/디지털과잉지수.csv", index=False, encoding="utf-8-sig")
df_final_2.to_csv("../5_데이터_처리_자료/무인행정수급불균형지수.csv", index=False, encoding="utf-8-sig")

In [47]:
df_base = df[["행정구역", "인구 수", "고령자 수", "독거 노인 수", "설치대수"]].copy()

# 분석용 종합 테이블 만들기
df_mix_final = (
    df_base
    .merge(df_ratio_1, on="행정구역", how="left")
    .merge(df_ratio_2[["행정구역", "행정수요비율"]], on="행정구역", how="left")
    .merge(df_norm_1, on="행정구역", how="left")
    .merge(df_norm_2[["행정구역", "행정수요비율(정규화)"]], on="행정구역", how="left")
    .merge(df_final_1, on="행정구역", how="left")
    .merge(df_final_2, on="행정구역", how="left")
)
df_mix_final.head(229)

,행정구역,인구 수,고령자 수,독거 노인 수,설치대수,고령인구구성비율,독거노인구성비율,무인행정공급비율,행정수요비율,고령인구구성비율(정규화),독거노인구성비율(정규화),무인행정공급비율(정규화),행정수요비율(정규화),디지털과잉지수,무인행정수급불균형지수
0,서울특별시 종로구,144486,27382,6118,28,0.189513,0.042343,0.000194,0.002789,0.260953,0.179584,0.221028,0.111776,0.010358,-0.109253
1,서울특별시 중구,126899,24360,6187,31,0.191964,0.048755,0.000244,0.002450,0.267902,0.229563,0.292546,0.097318,0.017992,-0.195228
2,서울특별시 용산구,203401,36473,7441,24,0.179316,0.036583,0.000118,0.003926,0.232034,0.134686,0.113682,0.160207,0.003553,0.046525
3,서울특별시 성동구,274688,48758,9647,30,0.177503,0.035120,0.000109,0.005302,0.226894,0.123282,0.101249,0.218809,0.002832,0.117560
4,서울특별시 광진구,345130,56766,11148,30,0.164477,0.032301,0.000087,0.006662,0.189954,0.101309,0.069680,0.276716,0.001341,0.207036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,경상남도 함양군,35073,13141,4225,7,0.374676,0.120463,0.000200,0.000677,0.786048,0.788486,0.229233,0.021832,0.142076,-0.207401
225,경상남도 거창군,58393,17867,5362,17,0.305978,0.091826,0.000291,0.001127,0.591232,0.565276,0.358885,0.041002,0.119943,-0.317883
226,경상남도 합천군,38950,16639,5749,21,0.427189,0.147599,0.000539,0.000752,0.934967,1.000000,0.710143,0.025019,0.663960,-0.685124
227,제주특별자치도 제주시,493908,80153,16897,52,0.162283,0.034211,0.000105,0.009534,0.183732,0.116196,0.095680,0.399020,0.002043,0.303339


In [48]:
df_mix_final.to_csv("../5_데이터_처리_자료/통합데이터.csv", index=False, encoding="utf-8-sig")